<a href="https://colab.research.google.com/github/SriMahith27/Summariser/blob/main/TextSummariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install and load spaCy
!pip install -q spacy
!python -m spacy download en_core_web_sm

import spacy
from heapq import nlargest
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Summarizer function
def summarize_text(text, num_sentences=3):
    doc = nlp(text)
    word_frequencies = {}
    stopwords = nlp.Defaults.stop_words

    for word in doc:
        if word.text.lower() not in stopwords and word.is_alpha:
            word_frequencies[word.text.lower()] = word_frequencies.get(word.text.lower(), 0) + 1

    max_freq = max(word_frequencies.values(), default=1)
    for word in word_frequencies:
        word_frequencies[word] /= max_freq

    sentence_scores = {}
    for sent in doc.sents:
        for word in sent:
            if word.text.lower() in word_frequencies:
                sentence_scores[sent] = sentence_scores.get(sent, 0) + word_frequencies[word.text.lower()]

    summarized = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    return ' '.join([sent.text for sent in summarized])

# UI Elements
text_input = widgets.Textarea(
    placeholder='Paste your text here or upload a .txt file...',
    description='Text:',
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': 'initial'}
)

sentence_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='Sentences:',
    continuous_update=False
)

summary_output = widgets.Textarea(
    description='Summary:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

upload_button = widgets.Button(description="Upload .txt File", button_style='info')

def upload_file(b):
    uploaded = files.upload()
    for fn in uploaded.keys():
        with open(fn, 'r', encoding='utf-8') as f:
            text_input.value = f.read()

upload_button.on_click(upload_file)

generate_button = widgets.Button(description="Generate Summary", button_style='success')

def on_click_generate(b):
    clear_output(wait=True)
    display(text_input, upload_button, sentence_slider, generate_button, summary_output)
    text = text_input.value
    if not text.strip():
        summary_output.value = "Please enter or upload some text."
        return
    summary = summarize_text(text, sentence_slider.value)
    summary_output.value = summary

generate_button.on_click(on_click_generate)

# Display UI
display(text_input, upload_button, sentence_slider, generate_button, summary_output)


Textarea(value='Climate change is one of the most pressing issues of our time. It refers to long-term shifts i…

Button(button_style='info', description='Upload .txt File', style=ButtonStyle())

IntSlider(value=3, continuous_update=False, description='Sentences:', max=10, min=1)

Button(button_style='success', description='Generate Summary', style=ButtonStyle())

Textarea(value='', description='Summary:', layout=Layout(height='150px', width='100%'), style=DescriptionStyle…